In [1]:
import numpy as np 
from timeit import timeit
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.stats import linregress
import scipy

from sympy import Symbol, Matrix, re
from sympy.solvers import solve
import easygdf
import scipy.io as scio

### Some functions for manipulating the input deck

In [2]:

def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()
    print("Line changed.")
    print("The line is now " +str(lines[line_num]) + '\n')
    return

def change_param(input_path, param_name, new_val):   
    with open(input_path, 'r') as gpt_in:

        lines =gpt_in.readlines()
        string = param_name
        n = len(string)
        for i in range(len(lines)):
            line = lines[i]
            #print('This line is '+str(line))
            #print('\n')
            if line[0:n] == string:
                print('The line is currently ' +str(line))
                print('\n')
                replace_line(input_path, i, string + ' ' + str(new_val)+' # updated value of '+ str(param_name) + '\n')
                
        print('\n')
        return

#### 4d Beam Matrix

In [4]:
#Define 4d correlation matrix
def get_mat_4d(screen):
    
    X = np.array(screen.get('x'))
    X = X - np.mean(X)
    Xp = np.array(screen.get('Bx'))
    Xp = Xp - np.mean(Xp)
    
    Y = np.array(screen.get('y'))
    Y = Y - np.mean(Y)
    Yp = np.array(screen.get('By'))
    Yp = Yp -np.mean(Yp)
    
    Z = np.array(screen.get('z'))
    G = np.array(screen.get('G'))
    Zp = np.array(screen.get('Bz'))
    Pz = G*Zp
    Px = G*Xp
    Py = G*Yp
    
    
    PS = np.array([X,Y,Xp,Yp]) #Phase space, state of the system at this time slice
    D = len(PS)
    corr = np.zeros((D,D))
    
    for i in range(len(PS)):
        corr[i,i] = np.mean(PS[i]**2)
        for j in range(i):
            #print("Evaluating S " + str(i) +str(j))
            corr[i,j] = np.mean(PS[i]*PS[j])
            corr[j,i] = corr[i,j]
            #print(corr[i,j])
            #print('\n')
            
    return corr

#### Matrix arithmetic

In [40]:
def matrix_product(lista):
    #Computes the matrix produce lista[n-1]*lista[n-2]*...*lista[0]
    result = np.transpose(lista[0])
    j = 1; #Counter
    while j < len(lista):
        result = np.matmul(np.transpose(lista[j]), result)
        #print("Result is ", result, " on iteration ", j, '\n')
        j +=1;
        
    return result

#### Functions for thin lens model

In [41]:
def matrixSkew(q):
    return Matrix([[1, 0, 0, 0], [0, 1, q, 0], [0, 0, 1, 0], [q, 0, 0, 1]])

def matrixDrift(d):
    return Matrix([[1, d, 0, 0], [0, 1, 0, 0], [0, 0, 1, d], [0, 0, 0, 1]])

def matrixSkewTriplet(q1, q2, q3, d0, d1, d2): 
    quad1 = matrixSkew(q1)
    quad2 = matrixSkew(q2)
    quad3 = matrixSkew(q3)
    
    drift0 = matrixDrift(d0)
    drift1 = matrixDrift(d1)
    drift2 = matrixDrift(d2)
    
    lattice = [quad3, drift2, quad2, drift1, quad1, drift0];            
    
    triplet = matrix_product(lattice)
                    

    return triplet

#### RTFB Transformation with screen as input, fixed quad distances

In [62]:


def RTFB_0(schiermo, d0 = 0.0413, d1 = 1.5, d2 = 2.3):
    
    SigMatrix = get_mat_4d(schiermo)
    
    xarray = schiermo.get('x')
    xparray = schiermo.get('Bx')
    yarray = schiermo.get('y')
    yparray = schiermo.get('By')
    zarray = schiermo.get('z')
    zparray = schiermo.get('Bz')
    Garray = schiermo.get('G') 


    varx = SigMatrix[0,0]
    corrxy = SigMatrix[0,1]
    corrxxp = SigMatrix[0,2]
    corrxyp = SigMatrix[0,3]

    vary = SigMatrix[1,1]
    corryxp = SigMatrix[1,2]
    corryyp = SigMatrix[1,3]

    varxp = SigMatrix[2,2]
    corrxpyp = SigMatrix[2,3]

    varyp = SigMatrix[3,3]
    avgG = np.mean(Garray)


    emixrms = np.sqrt(varx*varxp - corrxxp**2)
    emiyrms = np.sqrt(vary*varyp -corryyp**2)
    corr = np.abs( corrxy*corrxpyp - corrxyp*corryxp )
    emirrms = np.sqrt(avgG*avgG*emixrms*emiyrms -avgG*avgG*(corr) )
    
    beta = varx / emixrms 
    alpha = - corrxxp / emixrms
    gamma = (1 + alpha**2) / beta



    dT = d1 + d2

    q1p =  np.sqrt( (d1*alpha - beta - d1*dT*gamma + dT*alpha) / (-d1*dT*beta) )
    q1m = -np.sqrt( (d1*alpha - beta - d1*dT*gamma + dT*alpha) / (-d1*dT*beta) )

    q1 = q1m
    q2 = (beta - alpha*(d1+d2)) / (d1*d2*(1 - beta*q1))
    q3 = (alpha*d1*q1*q2 - gamma - q1 - q2) / (1 - alpha*(dT*q1 + d2*q2) + d1*d2*q2*(gamma + q1))


    mat = matrixSkewTriplet(q1, q2, q3, d0, d1, d2)

    newSigMatrix = np.array( matrix_product([mat,SigMatrix, np.transpose(mat)]) ).astype(float)


    #New correlations
    newvarx = newSigMatrix[0,0]
    newcorrxy = newSigMatrix[0,1]
    newcorrxxp = newSigMatrix[0,2]
    newcorrxyp = newSigMatrix[0,3]

    newvary = newSigMatrix[1,1]
    newcorryxp = newSigMatrix[1,2]
    newcorryyp = newSigMatrix[1,3]

    newvarxp = newSigMatrix[2,2]
    newcorrxpyp = newSigMatrix[2,3]

    newvaryp = newSigMatrix[3,3]

    enx = avgG*np.sqrt(np.abs(newvarx*newvarxp - newcorrxxp**2)) #avgG*np.sqrt(np.linalg.det(newSigMatrix[:2,:2]))
    eny = avgG*np.sqrt(np.abs(newvary*newvaryp -corryyp**2)) #np.sqrt(np.linalg.det(newSigMatrix[2:,2:]))
    e4d = avgG*np.sqrt(np.sqrt(np.linalg.det(newSigMatrix)))
    
    return [q1,q2,q3], eny/enx, [1e6*enx, 1e6*eny]

#### RTFB Transformation with the qi as input, fixed quad distance

In [58]:


def RTFB_1(schiermo, q1, q2, q3, d0 = 0.0413, d1 = 1.5, d2 = 2.3):
    
    #Get the correlation matrix
    SigMatrix = get_mat_4d(schiermo)
    
    xarray = schiermo.get('x')
    xparray = schiermo.get('Bx')
    yarray = schiermo.get('y')
    yparray = schiermo.get('By')
    zarray = schiermo.get('z')
    zparray = schiermo.get('Bz')
    Garray = schiermo.get('G') 


    varx = SigMatrix[0,0]
    corrxy = SigMatrix[0,1]
    corrxxp = SigMatrix[0,2]
    corrxyp = SigMatrix[0,3]

    vary = SigMatrix[1,1]
    corryxp = SigMatrix[1,2]
    corryyp = SigMatrix[1,3]

    varxp = SigMatrix[2,2]
    corrxpyp = SigMatrix[2,3]

    varyp = SigMatrix[3,3]
    avgG = np.mean(Garray)
    #d1 = 1.0; #m
    #d2 = 2.3; #m

    emixrms = np.sqrt(varx*varxp - corrxxp**2)
    emiyrms = np.sqrt(vary*varyp -corryyp**2)
    corr = np.abs( corrxy*corrxpyp - corrxyp*corryxp )
    emirrms = np.sqrt(avgG*avgG*emixrms*emiyrms -avgG*avgG*(corr) )
    #print(f'Initial Emittances (x,y): {1e6*avgG*emixrms} microns, {1e6*avgG*emiyrms} microns')
    beta = varx / emixrms 
    alpha = - corrxxp / emixrms
    gamma = (1 + alpha**2) / beta

    
    dT = d1 + d2


    mat = matrixSkewTriplet(q1, q2, q3, d0, d1, d2)

    newSigMatrix = np.array(matrix_product([mat, SigMatrix, np.transpose(mat)]) ).astype(float)


    #New correlations
    newvarx = newSigMatrix[0,0]
    newcorrxy = newSigMatrix[0,1]
    newcorrxxp = newSigMatrix[0,2]
    newcorrxyp = newSigMatrix[0,3]

    newvary = newSigMatrix[1,1]
    newcorryxp = newSigMatrix[1,2]
    newcorryyp = newSigMatrix[1,3]

    newvarxp = newSigMatrix[2,2]
    newcorrxpyp = newSigMatrix[2,3]

    newvaryp = newSigMatrix[3,3]

    enx = avgG*np.sqrt(np.abs(newvarx*newvarxp - newcorrxxp**2)) #avgG*np.sqrt(np.linalg.det(newSigMatrix[:2,:2]))
    eny = avgG*np.sqrt(np.abs(newvary*newvaryp -corryyp**2)) #np.sqrt(np.linalg.det(newSigMatrix[2:,2:]))
    e4d = avgG*np.sqrt(np.sqrt(np.linalg.det(newSigMatrix)))

    #print(f'RFTB Skew Triplet Settings: q1={q1}, q2={q2}, q3={q3}, d1={d1}, d2={d2}')

    #print(f'Minimum emittance: {min(enx,eny)*1e6} microns')
    #print(f'Maximum emittance: {max(enx,eny)*1e6} microns')
    #print(f'Normalized transverse emittance: {emirrms*1e6} microns')
    #print(f'4D Emittance: {e4d*1e6} microns')
    #print(f'Emittance Ratio: {max(enx,eny)/min(enx,eny)}')
    
    return [q1,q2,q3], eny/enx, [1e6*min(enx,eny), 1e6*max(enx,eny)]

### Now we use the thick lens model for a better estimate

In [76]:

L = 0.108;


def Rotate(phi):

	R = [[np.cos(phi), 0, np.sin(phi), 0], [0, np.cos(phi), 0,  np.sin(phi)], [-np.sin(phi), 0, np.cos(phi), 0], [0, -np.sin(phi), 0, np.cos(phi)]]

	return np.asarray(R)

def QskewFD(q, L): #First magnet focusing in xx', defocusing in yy'

	Kappa = np.abs(q/L)
    
	Qs11 = np.cos(L*np.sqrt(Kappa))
	Qs12 = np.sin(L*np.sqrt(Kappa))/np.sqrt(Kappa)
	Qs13 = 0
	Qs14 = 0

	Qs21 = -np.sqrt(Kappa)* np.sin(L*np.sqrt(Kappa))
	Qs22 = np.cos(L*np.sqrt(Kappa))
	Qs23 = 0
	Qs24 = 0

	Qs31 = 0
	Qs32 = 0
	Qs33 = np.cosh(L*np.sqrt(Kappa))
	Qs34 = np.sinh(L*np.sqrt(Kappa))/np.sqrt(Kappa)

	Qs41 = 0
	Qs42 = 0
	Qs43 = np.sqrt(Kappa)*np.sinh(L*np.sqrt(Kappa))
	Qs44 = np.cosh(L*np.sqrt(Kappa))

	Qs = np.array([[Qs11,Qs12,Qs13,Qs14],[Qs21,Qs22,Qs23,Qs24],[Qs31,Qs32,Qs33,Qs34],[Qs41,Qs42,Qs43,Qs44]]).astype(float)

	Qskew = matrix_product([Rotate(-np.pi/4.0), (np.asarray(Qs)), (Rotate(np.pi/4.0))] )
	
	return np.asarray(Qskew)


def QskewDF(q, L): #First magnet focusing in yy', defocusing in xx'

	Kappa = np.abs(q/L)

	Qs11 = np.cosh(L*np.sqrt(Kappa))
	Qs12 = np.sinh(L*np.sqrt(Kappa))/np.sqrt(Kappa)
	Qs13 = 0
	Qs14 = 0

	Qs21 = np.sqrt(Kappa)*np.sinh(L*np.sqrt(Kappa))
	Qs22 = np.cosh(L*np.sqrt(Kappa))
	Qs23 = 0
	Qs24 = 0

	Qs31 = 0
	Qs32 = 0
	Qs33 = np.cos(L*np.sqrt(Kappa))
	Qs34 = np.sin(L*np.sqrt(Kappa))/np.sqrt(Kappa)

	Qs41 = 0
	Qs42 = 0
	Qs43 = -np.sqrt(Kappa)* np.sin(L*np.sqrt(Kappa))
	Qs44 = np.cos(L*np.sqrt(Kappa))

	Qs = np.array([[Qs11,Qs12,Qs13,Qs14],[Qs21,Qs22,Qs23,Qs24],[Qs31,Qs32,Qs33,Qs34],[Qs41,Qs42,Qs43,Qs44]])

	Qskew = matrix_product([Rotate(-np.pi/4.0), Qs, Rotate(np.pi/4.0)])
	
	return np.asarray(Qskew)


def Drift(L):

	D = np.array([[1, L, 0, 0], [0, 1, 0, 0], [0, 0, 1, L], [0, 0, 0, 1]])
	
	return np.asarray(D)


def RTFB_thick(q1,q2,q3, d0, d2 ,d3 ):

	if (q2<0):
		RTFB = matrix_product([ QskewFD(np.abs(q3),L), Drift(d3), QskewDF(np.abs(q2),L), Drift(d2), QskewFD(np.abs(q1),L), Drift(d0) ])
	else:
		RTFB = matrix_product([ QskewDF(np.abs(q3),L), Drift(d3), (QskewFD(np.abs(q2),L)), Drift(d2), QskewDF(np.abs(q1),L), Drift(d0) ])

	return RTFB

In [45]:
def RTFB_thick_transform(schiermo, q1, q2, q3, d0, d1, d2):
    #Get the correlation matrix
    SigMatrix = get_mat_4d(schiermo)
    
    xarray = schiermo.get('x')
    xparray = schiermo.get('Bx')
    yarray = schiermo.get('y')
    yparray = schiermo.get('By')
    zarray = schiermo.get('z')
    zparray = schiermo.get('Bz')
    Garray = schiermo.get('G') 


    varx = SigMatrix[0,0]
    corrxy = SigMatrix[0,1]
    corrxxp = SigMatrix[0,2]
    corrxyp = SigMatrix[0,3]

    vary = SigMatrix[1,1]
    corryxp = SigMatrix[1,2]
    corryyp = SigMatrix[1,3]

    varxp = SigMatrix[2,2]
    corrxpyp = SigMatrix[2,3]

    varyp = SigMatrix[3,3]
    avgG = np.mean(Garray)
    #d1 = 1.0; #m
    #d2 = 2.3; #m

    emixrms = np.sqrt(varx*varxp - corrxxp**2)
    emiyrms = np.sqrt(vary*varyp -corryyp**2)
    corr = np.abs( corrxy*corrxpyp - corrxyp*corryxp )
    emirrms = np.sqrt(avgG*avgG*emixrms*emiyrms -avgG*avgG*(corr) )
    #print(f'Initial Emittances (x,y): {1e6*avgG*emixrms} microns, {1e6*avgG*emiyrms} microns')
    beta = varx / emixrms 
    alpha = - corrxxp / emixrms
    gamma = (1 + alpha**2) / beta

    
    dT = d1 + d2


    mat = RTFB_thick(q1, q2, q3, d0, d1, d2)

    newSigMatrix = np.array(matrix_product([mat,SigMatrix,np.transpose(mat)]) )

    #New correlations
    newvarx = newSigMatrix[0,0]
    newcorrxy = newSigMatrix[0,1]
    newcorrxxp = newSigMatrix[0,2]
    newcorrxyp = newSigMatrix[0,3]

    newvary = newSigMatrix[1,1]
    newcorryxp = newSigMatrix[1,2]
    newcorryyp = newSigMatrix[1,3]

    newvarxp = newSigMatrix[2,2]
    newcorrxpyp = newSigMatrix[2,3]

    newvaryp = newSigMatrix[3,3]

    enx = avgG*np.sqrt(np.abs(newvarx*newvarxp - newcorrxxp**2)) #avgG*np.sqrt(np.linalg.det(newSigMatrix[:2,:2]))
    eny = avgG*np.sqrt(np.abs(newvary*newvaryp -corryyp**2)) #np.sqrt(np.linalg.det(newSigMatrix[2:,2:]))
    e4d = avgG*np.sqrt(np.sqrt(np.abs(np.linalg.det(newSigMatrix))))

    
    return [q1,q2,q3], eny/enx, [1e6*enx, 1e6*eny]

#### Get starting values for the quad strengths

In [65]:
def init_guess(schiermo, d0 = 0.0413, d1 = 1.5, d2 = 2.3):
    
    SigMatrix = get_mat_4d(schiermo)
    
    xarray = schiermo.get('x')
    xparray = schiermo.get('Bx')
    yarray = schiermo.get('y')
    yparray = schiermo.get('By')
    zarray = schiermo.get('z')
    zparray = schiermo.get('Bz')
    Garray = schiermo.get('G') 


    varx = SigMatrix[0,0]
    corrxy = SigMatrix[0,1]
    corrxxp = SigMatrix[0,2]
    corrxyp = SigMatrix[0,3]

    vary = SigMatrix[1,1]
    corryxp = SigMatrix[1,2]
    corryyp = SigMatrix[1,3]

    varxp = SigMatrix[2,2]
    corrxpyp = SigMatrix[2,3]

    varyp = SigMatrix[3,3]
    avgG = np.mean(Garray)


    emixrms = np.sqrt(varx*varxp - corrxxp**2)
    emiyrms = np.sqrt(vary*varyp -corryyp**2)
    corr = np.abs( corrxy*corrxpyp - corrxyp*corryxp )
    emirrms = np.sqrt(avgG*avgG*emixrms*emiyrms -avgG*avgG*(corr) )
    
    beta = varx / emixrms 
    alpha = - corrxxp / emixrms
    gamma = (1 + alpha**2) / beta


    dT = d1 + d2

    q1p =  np.sqrt( np.abs((d1*alpha - beta - d1*dT*gamma + dT*alpha) / (-d1*dT*beta) ) )
    q1m = -np.sqrt( np.abs((d1*alpha - beta - d1*dT*gamma + dT*alpha) / (-d1*dT*beta) ) )

    q1 = q1m
    q2 = (beta - alpha*(d1+d2)) / (d1*d2*(1 - beta*q1))
    q3 = (alpha*d1*q1*q2 - gamma - q1 - q2) / (1 - alpha*(dT*q1 + d2*q2) + d1*d2*q2*(gamma + q1))


    mat = RTFB_thick(q1, q2, q3, d0, d1, d2)

    newSigMatrix = np.array(matrix_product([mat, SigMatrix, np.transpose(mat)]) )
    

    #New correlations
    newvarx = newSigMatrix[0,0]
    newcorrxy = newSigMatrix[0,1]
    newcorrxxp = newSigMatrix[0,2]
    newcorrxyp = newSigMatrix[0,3]

    newvary = newSigMatrix[1,1]
    newcorryxp = newSigMatrix[1,2]
    newcorryyp = newSigMatrix[1,3]

    newvarxp = newSigMatrix[2,2]
    newcorrxpyp = newSigMatrix[2,3]

    newvaryp = newSigMatrix[3,3]

    enx = avgG*np.sqrt(np.abs(newvarx*newvarxp - newcorrxxp**2)) #avgG*np.sqrt(np.linalg.det(newSigMatrix[:2,:2]))
    eny = avgG*np.sqrt(np.abs(newvary*newvaryp -corryyp**2)) #np.sqrt(np.linalg.det(newSigMatrix[2:,2:]))
    e4d = avgG*np.sqrt(np.sqrt(np.linalg.det(newSigMatrix)))


    
    return np.array([q1,q2,q3]), eny/enx, [1e6*enx, 1e6*eny]

In [66]:
print("Procedures are now available")

Procedures are now available


-1